In [183]:
import datetime
import ast
import json
from collections import Counter

import numpy as np
import pandas as pd
import string
import re

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestRegressor

import tensorflow as tf
from tensorflow import keras
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Flatten, Dense, concatenate
from sklearn.model_selection import train_test_split

In [184]:
df = pd.read_csv('new_cars.csv')

In [185]:
### Drop unncessary columns
columns_to_drop = ['link', 'stock_number', 'vin', 'deal_gauge', 'exterior_color', 'mpg', 'fuel_type', 'engine', 'price_history', 'vehicle_history', 'listing_title', 'interior_color', 'Open recall', 'Accidents or damage']
# add interior_color back once normalized
# add Open recall back once cleaned
# add Accidents or damage back once cleaned
df = df.drop(columns=columns_to_drop)

In [186]:
### Convert data types
boolean_columns = ['1-owner vehicle', 'Personal use only', 'Clean title']

for column in boolean_columns:
    df[column] = df[column].map({'Yes': True, 'No': False}).astype(bool)

In [187]:
# Check for NAs
missing_counts = df.isna().sum()
print(missing_counts[missing_counts > 0])

seller_name                  45
listed_date                1523
price_change               1523
price_change_percentage    1523
Cylinders                    74
Liters                      106
exterior_color_1            243
exterior_color_2           5990
low mpg                     427
high mpg                    427
dtype: int64


In [188]:
# Drop rows with a low number of missing values
columns_with_missing_values = ['seller_name', 'Cylinders', 'Liters', 'exterior_color_1', 'low mpg', 'high mpg']
df = df.dropna(subset=columns_with_missing_values)

# Drop exterior_color_2 since it only has 12 values
df = df.drop(columns=['exterior_color_2'])

In [189]:
# Drop missing price history data. ~25% of data.
columns_with_missing_values = ['listed_date', 'price_change', 'price_change_percentage']
df = df.dropna(subset=columns_with_missing_values)

In [190]:
df.head()

,listing_mileage,primary_price,drivetrain,transmission,seller_name,1-owner vehicle,Personal use only,Clean title,year,make,model,listed_date,price_change,price_change_percentage,Cylinders,Liters,Boosted,exterior_color_1,low mpg,high mpg
0,79497,17899,Front-wheel Drive,6-Speed Automatic,ETHIO Motors,True,False,True,2018,Hyundai,Santa Fe Sport 2.4L,2021-09-21,-2400.0,-11.823243,4.0,2.4,False,blue,21.0,27.0
1,80326,27900,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,True,True,2017,RAM,1500 Express,2022-03-15,-3085.0,-9.956431,8.0,5.7,False,silver,16.0,23.0
2,92959,18950,Front-wheel Drive,Automatic CVT,Custom Cars West,False,True,True,2019,Nissan,Kicks SR,2022-04-16,-1030.0,-5.155155,4.0,1.6,False,orange,31.0,36.0
3,59176,39950,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,False,True,2019,RAM,1500 Laramie,2022-06-01,-573.0,-1.414012,8.0,5.7,False,gray,19.0,24.0
4,135385,26950,Four-wheel Drive,-,Custom Cars West,False,True,True,2015,Jeep,Wrangler Unlimited Rubicon,2022-01-21,-3000.0,-10.016694,6.0,3.6,False,blue,16.0,21.0


In [191]:
df.shape

(3860, 20)

## DNN
### outcome variable: primary_price
### ocluded variables: price_change	price_change_percentage

In [192]:
# Drop 'price_change' and 'price_change_percentage' to avoid data leakage
columns_to_drop = ['price_change', 'price_change_percentage']
df = df.drop(columns=columns_to_drop)

## Categorical variables

In [193]:
categorical_columns = ['drivetrain', 'transmission', 'make', 'model', 'exterior_color_1']

# Check how many unique values there are in categorical_columns
for column in categorical_columns:
    unique_values = df[column].nunique()
    print(f"Unique values for {column}:", unique_values)

# Models has a large number of unique values
# transmission, make, model have a large number of unique values
# Consider embedding these instead of one-hot encode

Unique values for drivetrain: 6
Unique values for transmission: 83
Unique values for make: 47
Unique values for model: 1405
Unique values for exterior_color_1: 13


In [194]:
X = df.drop('primary_price', axis=1)  # Features
y = df['primary_price']  # Target

In [195]:
### One-hot Encode 
X = pd.get_dummies(X, columns=categorical_columns)

In [196]:
X.shape

(3860, 1566)

## Split data into 80% training set and 20% validation set.

In [197]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data
X_train = X_train.to_numpy()
X_val = X_val.to_numpy()
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()

# X_train = torch.tensor(X_train, dtype=torch.float32)
# X_val = torch.tensor(X_val, dtype=torch.float32)
# y_train = torch.tensor(y_train, dtype=torch.long)
# y_val = torch.tensor(y_val, dtype=torch.long)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.